In [1]:
from google.colab import drive
drive.mount( '/content/drive' )

Mounted at /content/drive


In [2]:
import time

**Libarary**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import CondensedNearestNeighbour
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import RepeatedEditedNearestNeighbours
from imblearn.under_sampling import AllKNN
from imblearn.under_sampling import NeighbourhoodCleaningRule
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks

In [5]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

**Train-Test Set**

In [8]:
df_card = pd.read_csv( '/content/drive/MyDrive/학술제 폴더/카드사기 탐지/카드사기 데이터/creditcard.csv')
col_name = [ 'V17', 'V12', 'V14', 'V16', 'V10', 'V11', 'V9', 'V4', 'V7', 'V26', 
            'V18', 'V21', 'V1', 'V27', 'V15', 'V8', 'V20', 'V3', 'Class' ]
df_card = df_card[col_name]

In [9]:
X, y = df_card.iloc[:,:-1], df_card.iloc[:, -1]

In [10]:
y.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [11]:
492*100/(284315)

0.17304750013189596

In [12]:
validation_ratio_1 = [ 0.00216, 0.0029, 0.0044, 0.0087]
validation_ratio_2 = [0.0086, 0.017, 0.0865, 0.17]

1,2는 각각 undersampling의 비율과 oversampling의 비율
undersampling 은 다수 클래스에서 0.2 0.4 0.6 0.8만큼 재거한 비율
oversampling은 소수 클래스를 5, 10, 50, 100배 만큼 함

In [13]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [14]:
classifier = [[LogisticRegression(), 'lreg'], [LinearSVC(C=0.001),'svc'], [RandomForestClassifier(max_depth=10),'rdf'], [KNeighborsClassifier(n_neighbors=3), 'knn']]

In [15]:
def Resampler(resampler, ratio):

  sampler = {
      'CC' : ClusterCentroids(sampling_strategy=ratio, random_state=0),
      'CNN': CondensedNearestNeighbour(random_state=0),
      'ENN': EditedNearestNeighbours(),
      'RENN': RepeatedEditedNearestNeighbours(),
      'AKNN': AllKNN(),
      'NCR': NeighbourhoodCleaningRule(),
      'RUS': RandomUnderSampler(sampling_strategy=ratio, random_state=0),
      'TL': TomekLinks,
      'ROS': RandomOverSampler(sampling_strategy=ratio, random_state=0),
      'SMOTE': SMOTE(random_state=0),
      'SVMSMOTE': SVMSMOTE(sampling_strategy=ratio, random_state=0),
  }
  
  return sampler[resampler]

원하는 Sampling을 선택하고 Resampler에 입력해서 돌린다

**UnderSampling**

In [17]:
score = []
for ratio in validation_ratio_1:
  n_iter = 0
  time_list = []
  for train_index, test_index in skf.split(X, y):
    n_iter += 1
    start = time.time()
    resampler = Resampler('RUS', ratio)
    X_res, y_res = resampler.fit_resample(X.iloc[train_index], y.iloc[train_index])
    time_list.append(time.time() - start)

    for clf in classifier:
      
      clf[0].fit(X_res, y_res)
      y_pred = clf[0].predict(X.iloc[test_index])
      score.append( [clf[1],ratio, accuracy_score(y_pred,  y.iloc[test_index]), precision_score(y_pred, y.iloc[test_index]),
                       recall_score(y_pred, y.iloc[test_index]), f1_score(y_pred, y.iloc[test_index])] )
      print(f1_score(y_pred, y.iloc[test_index]))

0.6705882352941176
0.6037735849056605
0.8314606741573033
0.8461538461538461
0.7542857142857142
0.7380952380952381
0.8695652173913043
0.8663101604278074
0.7073170731707316
0.6666666666666667
0.8539325842696629
0.8764044943820225
0.746987951807229
0.7124999999999999
0.858695652173913
0.8426966292134831
0.7303370786516853
0.6909090909090908
0.877005347593583
0.8648648648648648
0.7303370786516854
0.6785714285714286
0.8241758241758242
0.8387096774193549
0.7821229050279329
0.7337278106508875
0.8743169398907102
0.8617021276595745
0.7159090909090908
0.6951219512195121
0.8571428571428572
0.8681318681318683
0.7954545454545455
0.7745664739884393
0.858695652173913
0.8369565217391304
0.7604166666666667
0.7100591715976331
0.8795811518324608
0.8526315789473684
0.7643979057591622
0.7555555555555555
0.8279569892473118
0.8253968253968254
0.7934782608695652
0.7796610169491526
0.8663101604278074
0.8677248677248678
0.7311827956989247
0.7630057803468209
0.8404255319148936
0.858695652173913
0.827225130890052

**OverSampling**

In [ ]:
score = []
for ratio in validation_ratio_2:
  n_iter = 0
  time_list = []
  for train_index, test_index in skf.split(X, y):
    n_iter += 1
    start = time.time()
    resampler = Resampler('', ratio)
    X_res, y_res = resampler.fit_resample(X.iloc[train_index], y.iloc[train_index])
    time_list.append(time.time() - start)

    for clf in classifier:
      
      clf[0].fit(X_res, y_res)
      y_pred = clf[0].predict(X.iloc[test_index])
      score.append( [clf[1], ratio, accuracy_score(y_pred,  y.iloc[test_index]), precision_score(y_pred, y.iloc[test_index]),
                       recall_score(y_pred, y.iloc[test_index]), f1_score(y_pred, y.iloc[test_index])] )

**Evaluate**

In [20]:
def score_evaluate(score):
  length = int(len(score))
  score = score.astype(np.float64)
  acc, pre, rec, f1s = [], [], [], []
  for i in range(0,length,5):
    acc.append(score[i:i+5,1].mean())
    pre.append(score[i:i+5,2].mean())
    rec.append(score[i:i+5,3].mean())
    f1s.append(score[i:i+5,4].mean())
  
  return acc, pre, rec, f1s

In [21]:
score = np.array(score)

lreg_score_index = np.where(score == 'lreg')
svc_score_index = np.where(score == 'svc')
rdf_score_index = np.where(score == 'rdf')
knn_score_index = np.where(score == 'knn')
lreg_score = score[lreg_score_index[0]]
svc_score = score[svc_score_index[0]]
rdf_score = score[rdf_score_index[0]]
knn_score = score[knn_score_index[0]]
lreg_score = np.delete(lreg_score,0, axis=1)
svc_score = np.delete(svc_score,0, axis=1)
rdf_score = np.delete(rdf_score,0, axis=1)
knn_score = np.delete(knn_score,0, axis=1)

lreg_acc, lreg_pre, lreg_rec, lreg_f1s = score_evaluate(lreg_score)
svc_acc, svc_pre, svc_rec, svc_f1s = score_evaluate(svc_score)
rdf_acc, rdf_pre, rdf_rec, rdf_f1s = score_evaluate(rdf_score)
knn_acc, knn_pre, knn_rec, knn_f1s = score_evaluate(knn_score)

In [36]:
RUS_lreg=pd.DataFrame([lreg_acc, lreg_pre, lreg_rec, lreg_f1s],index = pd.Index(['accuracy', 'precision', 'recall', 'f1-score']), columns = ['-20%', '-40%','-60%','-80%']).transpose()
RUS_svc=pd.DataFrame([svc_acc, svc_pre, svc_rec, svc_f1s],index = pd.Index(['accuracy', 'precision', 'recall', 'f1-score']), columns = ['-20%', '-40%','-60%','-80%']).transpose()
RUS_rdf=pd.DataFrame([rdf_acc, rdf_pre, rdf_rec, rdf_f1s],index = pd.Index(['accuracy', 'precision', 'recall', 'f1-score']), columns = ['-20%', '-40%','-60%','-80%']).transpose()
RUS_knn=pd.DataFrame([knn_acc, knn_pre, knn_rec, knn_f1s],index = pd.Index(['accuracy', 'precision', 'recall', 'f1-score']), columns = ['-20%', '-40%','-60%','-80%']).transpose()

In [37]:
RUS_lreg

,accuracy,precision,recall,f1-score
-20%,0.999168,0.626036,0.854858,0.721903
-40%,0.999231,0.693135,0.835902,0.756848
-60%,0.999277,0.758215,0.811761,0.783257
-80%,0.999287,0.788683,0.797559,0.792537


In [38]:
RUS_svc

,accuracy,precision,recall,f1-score
-20%,0.999098,0.563039,0.867501,0.682389
-40%,0.999168,0.615914,0.862768,0.718409
-60%,0.999312,0.709421,0.868300,0.780417
-80%,0.999403,0.776500,0.864813,0.817852


In [39]:
RUS_rdf

,accuracy,precision,recall,f1-score
-20%,0.999547,0.794785,0.933568,0.858132
-40%,0.999544,0.804968,0.921279,0.858782
-60%,0.999494,0.815110,0.883768,0.847706
-80%,0.999410,0.823232,0.835429,0.828473


In [40]:
RUS_knn

,accuracy,precision,recall,f1-score
-20%,0.999551,0.794702,0.936041,0.859286
-40%,0.999515,0.804886,0.904585,0.851626
-60%,0.999477,0.806906,0.881060,0.842080
-80%,0.999343,0.817089,0.811364,0.812766


array안의 데이터를 이용해서 plot을 그린다

**Visualize**